In [1]:
import os

import pandas as pd
import numpy as np

from datetime import datetime

import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine

from flask import Flask, jsonify, render_template
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)


In [5]:
#################################################
# Database Setup
#################################################

app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:///db/maternal_health.sqlite"
db = SQLAlchemy(app)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(db.engine, reflect=True)

# Save references to each table
tbl_prm = Base.classes.pregnancy_related_mortality
tbl_ms = Base.classes.maternity_summary
tbl_mms = Base.classes.maternal_mortality_summary
tbl_mrs = Base.classes.maternity_random_sample
tbl_ufcs = Base.classes.us_female_census_summary

# Create dataframes for each table
stmt = db.session.query(tbl_prm).statement
df_prm = pd.read_sql_query(stmt, db.session.bind)

stmt = db.session.query(tbl_ms).statement
df_ms = pd.read_sql_query(stmt, db.session.bind)

stmt = db.session.query(tbl_mms).statement
df_mms = pd.read_sql_query(stmt, db.session.bind)

stmt = db.session.query(tbl_mrs).statement
df_mrs = pd.read_sql_query(stmt, db.session.bind)

stmt = db.session.query(tbl_ufcs).statement
df_ufcs = pd.read_sql_query(stmt, db.session.bind)



C:\Users\sjedael\Anaconda3\lib\site-packages\flask_sqlalchemy\__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [ ]:
@app.route("/")
def index():
    """Return the homepage."""
    return render_template("index.html")

@app.route("/regression/mrs/<predictors>/<response>")
# Need to look into being able to filter the table
def multi_regression(predictors, response):
    # Perform Multivariate Regression
    scale = StandardScaler()

    # Here we have 2 variables for multiple regression. If you just want to use one variable for simple linear regression, 
    #then use X = df['Interest_Rate'] for example. Alternatively, you may add additional variables within the brackets.

    column_list = predictors.split('&')

    X = df_mrs[column_list] 
    Y = df_mrs[response]

    X[column_list] = scale.fit_transform(X[column_list].as_matrix())

    est = sm.OLS(Y, X).fit()
    
    regression_results = {}
    regression_results['Dep. Variable'] = response
    regression_results['Model'] = 'OLS'
    regression_results['Method'] = 'Least Squares'
    regression_results['Date'] = datetime.now().strftime('%a, %d %b %Y')
    regression_results['Time'] = datetime.now().strftime('%H:%M:%S')
    regression_results['No. Observations'] = np.around(est.nobs,0)
    regression_results['Df Residuals'] = np.around(est.df_resid,0)
    regression_results['Df Model'] = est.df_model
    regression_results['Covariance Type'] = est.cov_type
    regression_results['R-Squared'] = np.around(est.rsquared,4)
    regression_results['Adj. R-Squared'] = np.around(est.rsquared_adj,4)
    regression_results['F-statistic'] = np.around(est.fvalue,4)
    regression_results['Prob (F-statistic)'] = np.around(est.f_pvalue,4)
    regression_results['Log-Likelihood'] = np.around(est.llf,4)
    regression_results['AIC'] = np.around(est.aic,4)
    regression_results['BIC'] = np.around(est.bic,4)
    
    params = []
    coef = []
    std_err = []
    t_values = []
    p_values = []
    low_conf = []
    upp_conf = []
    conf_int = est.conf_int().T
    
    for i in range(len(est.params)):
        params.append(est.params.index[i])
        coef.append(np.around(est.params[i],4))
        std_err.append(np.around(est.bse[i],4))
        t_values.append(np.around(est.tvalues[i],4))
        p_values.append(np.around(est.pvalues[i],4))
        low_conf.append(np.around(conf_int.iloc[0,i],4))
        upp_conf.append(np.around(conf_int.iloc[1,i],4))
    
    regression_results['Parameters'] = params
    regression_results['Coefficient'] = coef
    regression_results['Standard Error'] = std_err
    regression_results['T-Values'] = t_values
    regression_results['P-Values'] = p_values
    regression_results['Lower Confidence'] = low_conf
    regression_results['Upper Confidence'] = upp_conf
    
    return jsonify(regression_results)
    
@app.route("/maternalmorbidity")
def index():
    """Return the homepage."""
    return render_template("maternalmorbidity.html")


if __name__ == "__main__":
    app.run()

In [6]:
stmt = db.session.query(maternity_summary).statement
df = pd.read_sql_query(stmt, db.session.bind)
df.head()

,summary_id,delivery_year,race,hispanic,age_cohort,population,avg_prenatal_care_start_month,avg_num_prenatal_visits,avg_body_mass_index,prepregnancy_diabetes_count,...,delivery_not_stated_count,total_delivery_vaginal,total_delivery_cesarean,maternal_transfusion_count,perineal_laceration_count,ruptured_uterus_count,unplanned_hysterectomy_count,admit_intensive_care_count,total_morbidity_count,gestation_weeks
0,2009100,2009,"White, Non-Hispanic",Non-Hispanic,10-14,675,4.0620,9.7051,22.6286,0,...,0,518,157,2,10,0,0,0,12,38.1516
1,2009101,2009,"White, Non-Hispanic",Non-Hispanic,15-19,106689,3.4671,10.7057,24.1824,310,...,139,81720,24830,274,1229,14,10,116,1643,38.6266
2,2009102,2009,"White, Non-Hispanic",Non-Hispanic,20-24,330299,3.2287,11.0169,25.8291,1484,...,462,237508,92329,737,3405,60,55,351,4608,38.6422
3,2009103,2009,"White, Non-Hispanic",Non-Hispanic,25-29,435171,2.8998,11.4964,26.0610,2408,...,519,300814,133838,936,5529,104,107,409,7085,38.6103
4,2009104,2009,"White, Non-Hispanic",Non-Hispanic,30-34,364797,2.7541,11.6942,25.8672,2498,...,351,235765,128681,864,3884,111,160,414,5433,38.5164
